In [7]:
import torch
from sklearn.model_selection import train_test_split
import numpy as np

from sklearn.preprocessing import LabelEncoder

import torch.nn as nn
import torch.nn.functional as F

In [39]:
# Load the data
X = torch.load('torch_img_text.pt')
y = np.load('torch_styles.npy', allow_pickle=True)

# Create a label encoder object
le = LabelEncoder()

# Fit the label encoder and return encoded labels
y_encoded = le.fit_transform(y)

# Convert the numpy array to a PyTorch tensor
y_tensor = torch.tensor(y_encoded, dtype=torch.long)

# One-hot encode the labels
num_classes = 7  # As there are 7 unique labels (0 to 6)
y = torch.nn.functional.one_hot(y_tensor, num_classes=num_classes)
y = torch.tensor(y, dtype=torch.float)


# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


/tmp/ipykernel_15224/2838925515.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float)


In [40]:
X.shape, y.shape

(torch.Size([11688, 2048]), torch.Size([11688, 7]))

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 7)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)  # No activation here; softmax will be applied during inference
        return x

# Instantiate the model with input size 2048
input_size = 2048
model = SimpleNN(input_size)


In [57]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = np.Inf
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print('Early stopping triggered')
                self.early_stop = True

# Initialize EarlyStopping object
early_stopping = EarlyStopping(patience=5)

In [58]:
from torch.utils.data import TensorDataset, DataLoader

# Create TensorDatasets for training and validation
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

# Create DataLoaders for training and validation
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=64, shuffle=False)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.2f}')

    # Validation of the model
    model.eval()  # it will notify all your layers that you are in eval mode, that way, batchnorm or dropout layers will work in eval mode instead of training mode.
    val_loss = 0.0
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            # loss
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            # accuracy
            _, predicted = torch.max(outputs.data, 0)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        val_loss /= len(val_loader)
        accuracy = 100 * correct / total
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')
        
        # Early Stopping check
        early_stopping(val_loss)
        if early_stopping.early_stop:
            break


Epoch [1/1000], Step [100/147], Loss: 1.24
Validation Loss: 1.1109, Accuracy: 4.79%
Epoch [2/1000], Step [100/147], Loss: 1.09
Validation Loss: 1.0842, Accuracy: 0.09%
Epoch [3/1000], Step [100/147], Loss: 1.04
Validation Loss: 1.0613, Accuracy: 4.79%
Epoch [4/1000], Step [100/147], Loss: 1.11
Validation Loss: 1.0673, Accuracy: 7.01%
Epoch [5/1000], Step [100/147], Loss: 0.98
Validation Loss: 1.0668, Accuracy: 4.70%
Epoch [6/1000], Step [100/147], Loss: 1.09
Validation Loss: 1.0031, Accuracy: 7.70%
Epoch [7/1000], Step [100/147], Loss: 0.82
Validation Loss: 1.0632, Accuracy: 10.78%
Epoch [8/1000], Step [100/147], Loss: 0.82
Validation Loss: 1.0050, Accuracy: 10.78%
Epoch [9/1000], Step [100/147], Loss: 0.84
Validation Loss: 1.0878, Accuracy: 8.04%
Epoch [10/1000], Step [100/147], Loss: 0.72
Validation Loss: 0.9545, Accuracy: 10.78%
Epoch [11/1000], Step [100/147], Loss: 0.88
Validation Loss: 0.9370, Accuracy: 16.00%
Epoch [12/1000], Step [100/147], Loss: 0.95
Validation Loss: 0.9348, A

In [60]:
# Create TensorDataset for the test set
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoader for the test set
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)

        # Calculate loss if you want to report test loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 0)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
test_accuracy = 100 * correct / total
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.8519, Test Accuracy: 17.02%


In [80]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(4096 , 128)
        self.fc2 = nn.Linear(128, 7)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # Flatten the dimensions for the fully connected layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = SimpleCNN()



In [86]:
# Instantiate the CNN model
model = SimpleCNN()

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# EarlyStopping class as defined previously
early_stopping = EarlyStopping(patience=5)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
       
    for i, (inputs, labels) in enumerate(train_loader):
        # Reshape inputs if necessary
        inputs = inputs.view(-1, 2, 32, 32)  # Adjust this to match your data

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.2f}')

    # Validation of the model
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            # Reshape inputs if necessary
            inputs = inputs.view(-1, 2, 32, 32)  # Adjust this to match your data

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 0)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    accuracy = 100 * correct / total
    print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')

    # Early Stopping check
    early_stopping(val_loss)
    if early_stopping.early_stop:
        break


Epoch [1/1000], Step [100/147], Loss: 0.91
Validation Loss: 1.1208, Accuracy: 0.09%
Epoch [2/1000], Step [100/147], Loss: 0.89
Validation Loss: 1.0756, Accuracy: 4.88%
Epoch [3/1000], Step [100/147], Loss: 1.11
Validation Loss: 1.0086, Accuracy: 9.50%
Epoch [4/1000], Step [100/147], Loss: 0.83
Validation Loss: 0.9487, Accuracy: 9.41%
Epoch [5/1000], Step [100/147], Loss: 0.62
Validation Loss: 0.8940, Accuracy: 9.50%
Epoch [6/1000], Step [100/147], Loss: 0.88
Validation Loss: 0.8407, Accuracy: 16.77%
Epoch [7/1000], Step [100/147], Loss: 0.62
Validation Loss: 0.8203, Accuracy: 9.58%
Epoch [8/1000], Step [100/147], Loss: 0.74
Validation Loss: 0.7961, Accuracy: 14.80%
Epoch [9/1000], Step [100/147], Loss: 0.71
Validation Loss: 0.8079, Accuracy: 16.94%
Epoch [10/1000], Step [100/147], Loss: 0.62
Validation Loss: 0.7702, Accuracy: 14.46%
Epoch [11/1000], Step [100/147], Loss: 0.50
Validation Loss: 0.7544, Accuracy: 14.46%
Epoch [12/1000], Step [100/147], Loss: 0.45
Validation Loss: 0.7414, 

In [88]:
# Create TensorDataset for the test set
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoader for the test set
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.view(-1, 2, 32, 32)  # Adjust this to match your data
        outputs = model(inputs)

        # Calculate loss if you want to report test loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 0)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
test_accuracy = 100 * correct / total
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.7391, Test Accuracy: 14.63%
